In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import json

# Create a valid JSON object (Python dictionary)
data = {
	"name": "Alice",
	"age": 30,
	"is_student": False
}

# Convert to JSON string and print
json_str = json.dumps(data)
print(json_str)


{"name": "Alice", "age": 30, "is_student": false}
